In [28]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)


# %cd '/content/gdrive/MyDrive/LDS0_k282_Online_NguyenHoangThien/Project_1'

In [29]:
# !pip install underthesea
# !pip install demoji
# !pip install pyvi

In [15]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string

In [16]:
##LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
english_lst = file.read().split('\n')
english_dict = {}
for line in english_lst:
    key, value = line.split('\t')
    english_dict[key] = str(value)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [13]:
emoji_dict['😂']

'cười'

In [17]:
teen_dict['ko']

'không'

In [18]:
def process_text(text, emoji_dict, teen_dict, wrong_lst):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### CONVERT EMOJICON
        sentence = ''.join(emoji_dict[word]+' ' if word in emoji_dict else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(teen_dict[word] if word in teen_dict else word for word in sentence.split())
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        ###### DEL wrong words   
        sentence = ' '.join('' if word in wrong_lst else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '                    
    document = new_sentence  
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [24]:
example = '''1m6 50kg size M khá vừa vặn nhưng ko có được cơ nên nhìn cx thường 😊. Nâu với trắng vải dày hơn đỏ, trắng với đỏ cổ cao hơn nâu một tí, trắng đẹp 👍. 
Vai áo chưa đẹp lắm. Khá ổn so với giá tiền'''

In [25]:
document = process_text(example, emoji_dict, teen_dict, wrong_lst)
document

'khá vừa vặn nhưng không có được cơ nên nhìn cũng thường cười. nâu với trắng vải dày hơn đỏ trắng với đỏ cổ hơn nâu một tí trắng đẹp thích. áo chưa đẹp lắm. khá ổn với giá tiền.'

In [26]:
# Chuẩn hóa unicode tiếng việt
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [27]:
document = covert_unicode(document)
document

'khá vừa vặn nhưng không có được cơ nên nhìn cũng thường cười. nâu với trắng vải dày hơn đỏ trắng với đỏ cổ hơn nâu một tí trắng đẹp thích. áo chưa đẹp lắm. khá ổn với giá tiền.'

In [28]:
# có thể bổ sung thêm các từ: chẳng, chả...
def process_special_word(text):
    new_text = ''
    text_lst = text.split()
    i= 0
    if ('không' or 'chẳng') in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
            #print(word)
            #print(i)
            if  word == 'không' or word == 'không được' or word == 'chẳng':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    else:
        new_text = text
    return new_text.strip()

In [29]:
x = process_special_word(document)
x

'khá vừa vặn nhưng không_có được cơ nên nhìn cũng thường cười. nâu với trắng vải dày hơn đỏ trắng với đỏ cổ hơn nâu một tí trắng đẹp thích. áo chưa đẹp lắm. khá ổn với giá tiền.'

In [30]:
def process_postag_thesea(text):
    new_document = ''
    for sentence in sent_tokenize(text):
        sentence = sentence.replace('.','')
        ###### POS tag
        lst_word_type = ['A','AB','V','VB','VY','R']
        sentence = ' '.join( word[0] if word[1].upper() in lst_word_type else '' for word in pos_tag(process_special_word(word_tokenize(sentence, format="text"))))
        new_document = new_document + sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [31]:
document = process_postag_thesea(document)
document

'khá vừa_vặn không_có được cơ nhìn cũng thường cười nâu trắng dày hơn đỏ trắng đỏ cổ hơn trắng đẹp thích chưa đẹp lắm khá ổn'

In [32]:
def remove_stopword(text, stopwords):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [33]:
document = remove_stopword(document, stopwords_lst)
document

'vừa_vặn cười nâu trắng dày đỏ trắng đỏ cổ trắng đẹp thích đẹp lắm ổn'

In [34]:
df = pd.read_csv('Products_Shopee_comments.csv')

In [35]:
df.shape

(1903157, 6)

In [44]:
for cate in df.category.unique():
    non_null = df[df.category == cate]['comment'].count()
    print(f"{cate}: has {non_null} values not null")

Điện Thoại & Phụ Kiện: has 61233 values not null
Máy Ảnh & Máy Quay Phim: has 3659 values not null
Máy Tính & Laptop: has 10471 values not null
Mẹ & Bé: has 74865 values not null
Nhà Cửa & Đời Sống: has 11853 values not null
Sắc Đẹp: has 871 values not null
Thiết Bị Điện Tử: has 10772 values not null
Thời Trang Nam: has 537950 values not null
nan: has 0 values not null
Thời Trang Nữ: has 229628 values not null


In [62]:
df = df.loc[df['category'] == 'Thời Trang Nữ']

In [63]:
df.dropna(axis=0, inplace=True, subset=['comment'])

In [64]:
df.category.value_counts()

Thời Trang Nữ    229628
Name: category, dtype: int64

In [65]:
df.shape

(229628, 6)

In [66]:
df.head()

,product_id,category,sub_category,user,rating,comment
1319050,0,Thời Trang Nữ,"Áo khoác, Áo choàng & Vest",q*****3,5,sản_phẩm đóng_gói đẹp shop tốt
1319052,0,Thời Trang Nữ,"Áo khoác, Áo choàng & Vest",0*****y,5,sản_phẩm đóng_gói đẹp shop tốt
1319054,0,Thời Trang Nữ,"Áo khoác, Áo choàng & Vest",minhanhnguyen807,5,sản_phẩm đóng_gói đẹp shop tốt
1319058,0,Thời Trang Nữ,"Áo khoác, Áo choàng & Vest",kimngnnguyn183,5,sản_phẩm đóng_gói đẹp shop tốt
1319062,0,Thời Trang Nữ,"Áo khoác, Áo choàng & Vest",.*****0,5,sản_phẩm đóng_gói đẹp shop tốt


In [67]:
df['comment'] = df['comment'].apply(lambda x: process_text(x, emoji_dict, teen_dict, wrong_lst))

In [68]:
df['comment'] = df['comment'].apply(lambda x: covert_unicode(x))

In [69]:
df['comment'] = df['comment'].apply(lambda x: process_special_word(x))

In [70]:
df['comment'] = df['comment'].apply(lambda x: process_postag_thesea(x))

In [71]:
df['comment'] = df['comment'].apply(lambda x: remove_stopword(x, stopwords_lst))

In [72]:
df.to_csv('preprocessed_df_3.csv', index=False)

In [ ]:
def word_cloud()